### TLS session resuse with session ticket

In [ ]:

from scapy_ssl_tls.ssl_tls import *
host = ("10.105.5.75", 443)
version = TLSVersion.TLS_1_1
#cipher = TLSCipherSuite.ECDHE_RSA_WITH_AES_256_CBC_SHA
cipher = TLSCipherSuite.RSA_WITH_AES_256_CBC_SHA
ticket = ""
master_secret = ""


with TLSSocket(socket.socket(), client=True) as tls_socket:
    tls_socket.connect(host)
    tls_ctx = tls_socket.tls_ctx

    pkt = TLSRecord() / TLSHandshake() / TLSClientHello(version=version, cipher_suites=[cipher],
                                                        extensions=[TLSExtension() / TLSExtSessionTicketTLS(data="")])
    tls_socket.sendall(pkt)
    tls_socket.recvall()
    client_key_exchange = TLSRecord(version=version) / TLSHandshake() / tls_ctx.get_client_kex_data()
    client_ccs = TLSRecord(version=version) / TLSChangeCipherSpec()
    tls_socket.sendall(TLS.from_records([client_key_exchange, client_ccs]))
    tls_socket.sendall(to_raw(TLSFinished(), tls_ctx))
    server_finished = tls_socket.recvall()
    ticket = server_finished[TLSSessionTicket].ticket # thsi si a laer to the packet similar to pkt[IP].src
    tls_socket.sendall(to_raw(TLSPlaintext(data="GET / HTTP/1.1\r\nHOST: localhost\r\n\r\n"), tls_ctx))
    tls_socket.recvall()
    master_secret = tls_ctx.crypto.session.master_secret

print("First session context: %s" % tls_ctx)

with TLSSocket(socket.socket(), client=True) as tls_socket:
    tls_socket.connect(host)
    tls_ctx = tls_socket.tls_ctx
    tls_socket.tls_ctx.resume_session(master_secret)

    pkt = TLSRecord() / TLSHandshake() / TLSClientHello(version=version, cipher_suites=[cipher],
                                                        extensions=[TLSExtension() / TLSExtSessionTicketTLS(data=ticket)])
    tls_socket.sendall(pkt)
    resp = tls_socket.recvall()
    tls_socket.sendall(TLSRecord(version=version) / TLSChangeCipherSpec())
    tls_socket.sendall(to_raw(TLSFinished(), tls_ctx))
    tls_socket.sendall(to_raw(TLSPlaintext(data="GET / HTTP/1.1\r\nHOST: localhost\r\n\r\n"), tls_ctx))
    tls_socket.recvall()

print("Resumed session context: %s" % tls_ctx)


### client authenticated TLS session

In [ ]:

from __future__ import with_statement
from __future__ import print_function
import os
import socket
import sys

from Cryptodome.Hash import SHA256
from scapy_ssl_tls.ssl_tls import *
from scapy.all import *



def do_tls_mutual_auth(host):
    hand=open('/home/ftp/ia.crt')
    client_certy=hand.read()
    certificate = TLSCertificate(data=client_certy)

    tls_version = TLSVersion.TLS_1_2

    socket_ = socket.socket()
    tls_socket = TLSSocket(socket_, client=True)
    tls_socket.connect(host)
    tls_socket.tls_ctx.client_ctx.load_rsa_keys_from_file('/home/ftp/ia.key')

    client_hello = TLSRecord(version=tls_version) / \
    TLSHandshakes(handshakes=[TLSHandshake() /TLSClientHello(version=tls_version, cipher_suites=[TLSCipherSuite.ECDHE_RSA_WITH_AES_256_CBC_SHA, ])])
    tls_socket.sendall(client_hello) 
    server_hello = tls_socket.recvall()
    server_hello.show()


    client_cert = TLSRecord(version=tls_version) / TLSHandshakes(handshakes=[TLSHandshake()/TLSCertificate(data=client_certy)])
    client_key_exchange = TLSRecord(version=tls_version) / TLSHandshakes(handshakes=[TLSHandshake()/ tls_socket.tls_ctx.get_client_kex_data()]) 
    p = TLS.from_records([client_cert, client_key_exchange])
    tls_socket.sendall(p)

    
    sig = tls_socket.tls_ctx.get_client_signed_handshake_hash(SHA256.new())
    # sig = sig[:128] + chr(ord(sig[128]) ^ 0xff) + sig[129:]
    client_cert_verify = TLSRecord(version=tls_version) / TLSHandshakes(handshakes=[TLSHandshake()/TLSCertificateVerify(alg=sig_hash_alg,sig=sig)]) 
    tls_socket.sendall(client_cert_verify)

    client_ccs = TLSRecord(version=tls_version) / TLSChangeCipherSpec()
    tls_socket.sendall(client_ccs)
    tls_socket.sendall(to_raw(TLSFinished(), tls_socket.tls_ctx))
    server_finished = tls_socket.recvall()
    server_finished.show()

    tls_socket.sendall(to_raw(TLSPlaintext(data="GET / HTTP/1.1\r\nHOST: localhost\r\n\r\n"), tls_socket.tls_ctx))
    resp = tls_socket.recvall()
    print("Got response from server")
    resp.show()
    print(tls_socket.tls_ctx)


if __name__ == "__main__":
    server=('10.105.5.75', 443)
    do_tls_mutual_auth(server)

### server emulation with control over TLS record size 
##### used to recreate scenarios with 1 byte of http payload in a ssl record that triggers crashes. 

In [ ]:


from __future__ import with_statement
from __future__ import print_function
import os
import socket
import sys
import time
from scapy.all import *
from scapy_ssl_tls.ssl_tls import *



tls_version = TLSVersion.TLS_1_2
#tls_version = TLSVersion.PROTOCOL_DTLS_1_0_OPENSSL_PRE_0_9_8f

def tls_hello(sock):
    client_hello = TLSRecord(version=tls_version) / TLSHandshake() /\
                   TLSClientHello(version=tls_version, compression_methods=[TLSCompressionMethod.NULL, ],
                                  cipher_suites=[TLSCipherSuite.RSA_WITH_AES_256_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_RC4_128_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_DSS_WITH_AES_128_CBC_SHA, ])
    sock.sendall(client_hello)
    server_hello = sock.recvall()
    server_hello.show()


def tls_client_key_exchange(sock):
    global flag
    client_key_exchange = TLSRecord(version=tls_version) / TLSHandshake() / sock.tls_ctx.get_client_kex_data()
    client_ccs = TLSRecord(version=tls_version) / TLSChangeCipherSpec()
    sock.sendall(client_key_exchange)
    sock.sendall(client_ccs)
    time.sleep(0.1) # without this it was sending change cipher spec and finish together in one ssl payload
#    sock.sendall(TLS.from_records([client_key_exchange, client_ccs])) # added these two records in one ssl payload
    sock.sendall(to_raw(TLSFinished(), sock.tls_ctx))
    print("printing client finish") 
    server_finished = sock.recvall()
    flag=1
    server_finished.show()

def ttoraw(pkt,tls_ctx):
    from scapy.layers.ssl_tls_crypto import *
    capp=[]
    capprec=[]
    for p in pkt:
        crypto_mod = CryptoContainer(tls_ctx, p)
        ciphertxt=crypto_mod.encrypt()
        capprec.append(TLSRecord(version=tls_ctx.params.negotiated.version, content_type='application_data') / ciphertxt)
    return capprec

def control_recordlengthstack(pkt, tls_ctx, size=2**14):
    payload = str(pkt)
    payloads = [payload[i: i + size] for i in range(0, len(payload), size)]
    fragments = []
    apprecords=ttoraw(payloads, tls_ctx)
    stack = TLS.from_records(apprecords)
    return stack

def tls_client(ip, priv_key=None):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.connect(ip)
        sock = TLSSocket(sock, client=True) 
        print("Connected to server: %s" % (ip,))
    except socket.timeout as te:
        print("Failed to open connection to server: %s" % (ip,), file=sys.stderr)
    else:
        tls_hello(sock)
        tls_client_key_exchange(sock)
        print("Finished handshake. Sending application data (GET request)")
        data1='GET / HTTP/1.1\r\nHost: 10.105.5.75\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nAccept-Language: en-US,en;q=0.5\r\nAccept-Encoding: gzip, deflate\r\nCookie: PHPSESSID=297dcfe25dc5f26f5a5436a142126c02\r\nDNT: 1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\n\r\n'
        data3='GET / HTTP/1.1\r\nHOST: 10.105.5.75\r\n\r\n'
        data2="A"*(2**11)
        fr=control_recordlengthstack(data3, sock.tls_ctx,10)     
        sock.sendall(fr)
        resp = sock.recvall()
    finally:
        sock.close()

if __name__ == "__main__":
    global flag
    flag=0
    server = ("10.105.5.75", 443)
    tls_client(server)

### control padding in TLS messages
##### used in recreation of bugs in handling  tlsfinished encrypted message with xl padding

In [ ]:

import argparse
import binascii
import StringIO
from __future__ import with_statement
from __future__ import print_function
import os
import socket
import sys
import time
from scapy.all import *
from scapy_ssl_tls.ssl_tls import *
from scapy.layers.ssl_tls_crypto import *



tls_version = TLSVersion.TLS_1_2


def tls_hello(sock):
    client_hello = TLSRecord(version=tls_version,content_type = 22) / TLSHandshake() /\
                   TLSClientHello(version=tls_version, compression_methods=[TLSCompressionMethod.NULL, ],
                                  cipher_suites=[TLSCipherSuite.RSA_WITH_AES_256_CBC_SHA, ])
    client_hello.show()
    sock.sendall(client_hello)
    server_hello = sock.recvall()

def tls_client_key_exchange(sock):
    global flag
    client_key_exchange = TLSRecord(version=tls_version) / TLSHandshake() / sock.tls_ctx.get_client_kex_data()
    client_ccs = TLSRecord(version=tls_version) / TLSChangeCipherSpec()
    sock.sendall(client_key_exchange)
    sock.sendall(client_ccs)
    time.sleep(0.1) 
    v=sock.tls_ctx.get_verify_data()
    sock.sendall(to_raw(TLSRecord(version=tls_version) / TLSFinished(), sock.tls_ctx, pre_encrypt_hook=modify_padding))
    print("printing client finish") 
    server_finished = sock.recvall()
    flag=1

def modify_padding(crypto_container):
    output = StringIO.StringIO()
    padding = crypto_container.padding
    regpad=(binascii.hexlify(padding))
    val=91
    for _ in xrange(val):
        output.write('%02x' % val)
    bin=binascii.unhexlify(output.getvalue())
    bigpad=(binascii.hexlify(bin))
    crypto_container.padding = bin
    return crypto_container
    
def ttoraw(pkt,tls_ctx):
    
    capp=[]
    capprec=[]
    for p in pkt:
        crypto_mod = CryptoContainer(tls_ctx, p)
        print(tls_ctx.params.negotiated.version)
        ciphertxt=crypto_mod.encrypt()
        print("length of ciphertext",len(ciphertxt))
        capprec.append(TLSRecord(version=tls_ctx.params.negotiated.version, content_type='application_data') / ciphertxt)
    return capprec
    
def control_recordlengthstack(pkt, tls_ctx, size=2**14):
    payload = str(pkt)
    payloads = [payload[i: i + size] for i in range(0, len(payload), size)]
    fragments = []
    apprecords=ttoraw(payloads, tls_ctx)
    stack = TLS.from_records(apprecords)
    return stack

def tls_client(ip, priv_key=None):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind(('172.168.1.10', 0))
        sock.connect(ip)
        sock = TLSSocket(sock, client=True) 
        print("Connected to server: %s" % (ip,))
    except socket.timeout as te:
        print("Failed to open connection to server: %s" % (ip,), file=sys.stderr)
    else:
        tls_hello(sock)
        tls_client_key_exchange(sock)
        print("Finished handshake. Sending application data (GET request)")
        data1='GET / HTTP/1.1\r\nHost: 10.105.5.75\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nAccept-Language: en-US,en;q=0.5\r\nAccept-Encoding: gzip, deflate\r\nCookie: PHPSESSID=297dcfe25dc5f26f5a5436a142126c02\r\nDNT: 1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\n\r\n'
        fr=control_recordlengthstack(data1, sock.tls_ctx)     
        sock.sendall(fr)
        resp = sock.recvall()

    finally:
        sock.close()

parser = argparse.ArgumentParser()
parser.add_argument("Server_IP", type=str,
                    help="Destination IP address you want to send traffic to")
parser.add_argument("-v", "--verbose", action="store_true",
                    help="increase output verbosity")
#args = parser.parse_args()
#dest = str(args.Server_IP)
#server = (dest, 443)
server =("10.101.88.43",443)
tls_client(server)

### create ssl handshake failures
#### the case below as a particular scenario of RST after client hello

In [ ]:

from __future__ import with_statement
from __future__ import print_function
import os
import socket,select
import sys
import time
from scapy.all import *
from scapy_ssl_tls.ssl_tls import *



tls_version = TLSVersion.TLS_1_1


def tls_hello(sock):
    client_hello = TLSRecord(version=tls_version) / TLSHandshake() /\
                   TLSClientHello(version=tls_version, compression_methods=[TLSCompressionMethod.NULL, ],
                                  cipher_suites=[TLSCipherSuite.RSA_WITH_AES_256_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_RC4_128_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_DSS_WITH_AES_128_CBC_SHA, ])
#    print (type(client_hello), client_hello, repr(client_hello))
    sock.sendall(client_hello)
    server_hello = sock.recvall()
    server_hello.show()
    
def tls_client_key_exchange(sock):
    global flag
    client_key_exchange = TLSRecord(version=tls_version) / TLSHandshake() / sock.tls_ctx.get_client_kex_data()
    
    client_ccs = TLSRecord(version=tls_version) / TLSChangeCipherSpec()
    sock.sendall(client_key_exchange)
    sock.sendall(client_ccs)
    time.sleep(0.1) 

    sock.sendall(to_raw(TLSFinished(), sock.tls_ctx))
    server_finished = sock.recvall()
    flag=1
    
def tls_client(ip, priv_key=None):
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    l_onoff = 1                                                                                                                                                           
    l_linger = 0
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_LINGER, struct.pack('ii', l_onoff, l_linger))
    sock.bind(("10.107.246.199",0))
    try:
        sock = TLSSocket(sock, client=True) 
        sock.connect(ip)
        print (sock.fileno())

        print("Connected to server: %s" % (ip,))
    except socket.timeout as te:
        print("Failed to open connection to server: %s" % (ip,), file=sys.stderr)
    else:
        
        tls_hello(sock)
        print (sock.tls_ctx)
        tls_client_key_exchange(sock)
        alert = TLSRecord(version=tls_version) / TLSAlert(level=TLSAlertLevel.FATAL, description=0x30)
        sock.sendall(to_raw(alert, sock.tls_ctx))
    finally:
        sock.close()

if __name__ == "__main__":
    global flag
    flag=0
    server = ("10.105.5.75", 443)
    tls_client(server)

### client hello mutations -malformed len field value

In [ ]:
#clienthello mutated length fileds to create panic in buf_mem_grow and malloc

# create handshake failure , rst after client hello
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import with_statement
from __future__ import print_function
import os
import socket
import sys
import time
from scapy.all import *
from scapy_ssl_tls.ssl_tls import *

#TLSExtension() / TLSExtServerNameIndication(server_names= [TLSServerName(data="a",length=16386)])

'''extensions=[TLSExtension()/ \
                                  TLSExtALPN(protocol_name_list= \
                                                    [TLSALPNProtocol(data="http/1.1"),
                                                     TLSALPNProtocol(data="http/1.3"),
                                                     TLSALPNProtocol(data="\x00htt\x01%sp/1.1"),
                                                     ])],)'''


tls_version = TLSVersion.TLS_1_1
#, length=16386

def tls_hello(sock):
    client_hello = TLSRecord(version=tls_version, length=1) / TLSHandshake() /\
                   TLSClientHello(version=tls_version, 
                                  compression_methods=[TLSCompressionMethod.NULL, ],
                                  cipher_suites=[TLSCipherSuite.RSA_WITH_AES_256_CBC_SHA, ],
                                  extensions=[TLSExtension()/TLSExtALPN(protocol_name_list= 
                                                    [TLSALPNProtocol(data="http/1.1"),
                                                     TLSALPNProtocol(data="http/1.3"),
                                                     TLSALPNProtocol(data="\x00htt\x01%sp/1.1")
                                                     ])])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.RSA_WITH_RC4_128_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_RSA_WITH_AES_128_CBC_SHA, ])
                                  # cipher_suites=[TLSCipherSuite.DHE_DSS_WITH_AES_128_CBC_SHA, ])
    client_hello.show()
    sock.sendall(client_hello)
    server_hello = sock.recvall()
    server_hello.show()
    
def tls_client_key_exchange(sock):
    global flag
    client_key_exchange = TLSRecord(version=tls_version) / TLSHandshake() / sock.tls_ctx.get_client_kex_data()
    client_ccs = TLSRecord(version=tls_version) / TLSChangeCipherSpec()
    sock.sendall(client_key_exchange)
    sock.sendall(client_ccs)
    time.sleep(0.1) 

    sock.sendall(to_raw(TLSFinished(), sock.tls_ctx))
    server_finished = sock.recvall()
    flag=1
    
def tls_client(ip, priv_key=None):
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    l_onoff = 1                                                                                                                                                           
    l_linger = 0
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_LINGER, struct.pack('ii', l_onoff, l_linger))
    sock.bind(("10.107.246.199",0))
    try:
        sock.connect(ip)
        sock = TLSSocket(sock, client=True) 
        print("Connected to server: %s" % (ip,))
    except socket.timeout as te:
        print("Failed to open connection to server: %s" % (ip,), file=sys.stderr)
    else:
        print (sock.tls_ctx)
        tls_hello(sock)
        tls_client_key_exchange(sock)
        alert = TLSRecord(version=tls_version) / TLSAlert(level=TLSAlertLevel.FATAL, description=0x30)
        sock.sendall(to_raw(alert, sock.tls_ctx))
    finally:
        sock.close()

if __name__ == "__main__":
    global flag
    flag=0
    server = ("10.107.251.192", 443)
    tls_client(server)